In [ ]:
import pickle as pickle
import numpy as np
import matplotlib.pyplot as plt
from pprint import pprint as pp
import math as math
from matplotlib.ticker import FuncFormatter, MaxNLocator
import pandas as pd
from cosmocalc import cosmocalc
import scipy
from scipy.optimize import fsolve

import sympy
from sympy.solvers import nsolve
from sympy import Symbol

In [3]:
f=open('cal_sensitivity_data.p','rb')
cal_sigma=pickle.load(f, encoding='latin1') #dictionary with sensitivities for each frequency bin for each calibrator
f.close()

#cal_sig= np.array(list(cal_sigma.items()) #convert dictionary to array
                
N_freq=len(cal_sigma['3c84']) 
N_sources=len(cal_sigma.keys())
cal_sig_array =np.zeros([N_freq,N_sources]) 

for x,source in enumerate(cal_sigma.keys()):
    for y in range(N_freq):
        cal_sig_array[y,x]=cal_sigma[source][y]
        
    
cal_sig_array[np.where(cal_sig_array!=cal_sig_array)]=1000000.

sensitivity_threshold=np.geomspace(.1,100,31) #create sensitivity thresholds 

#binStruct= {}
#binStruct['sensitivity']=np.zeros(len(sensitivity_threshold))
#binStruct['bin_count']=np.zeros(len(sensitivity_threshold))
N_freq=len(cal_sigma['3c84'])
N_threshold=len(sensitivity_threshold)
binned_array = np.zeros(shape=(N_freq,N_threshold))

 #first loop is going thru each frequency bin    
for i in range(N_freq):
  
    #second loop is going thru each sensitivity threshold 
    for j in range(N_threshold):
        binned_array[i,j]=np.sum(cal_sig_array[i]*1500<=sensitivity_threshold[j]) #factor of 1500 to take into account 300 kn/s and 5 sigma detection



In [ ]:
cal_sig_array.shape  #(sensitivity for each sources (344?) for each freq bin (181) ? 
print(cal_sig_array[

In [ ]:
len(cal_sigma['3c84'])
binned_array.shape

In [ ]:
#181 freq bins
#344 sources
#sensitivity threshold in Jy

In [ ]:
 print(cal_sig_array[180,340])

# PLOTS

In [ ]:

fig = plt.figure()
ax = fig.add_subplot(1, 1,1)
plot = ax.imshow(np.transpose(np.log10(binned_array)),extent=[190,371,100,.1],aspect='auto')

fig.colorbar(plot)
ax.set_yticklabels([.1,.5,3,13,63,100])


#plt.yscale('log')
#extent=[horizontal_min,horizontal_max,vertical_min,vertical_max].

y_label=['{:.2f}'.format(x) for x in sensitivity_threshold]
print(y_label)

#plt.xtitle('Sensitivity Thresholds')
#plt.ytitle('Frequency')
#plt.Axes.set_yticklabels('.1.003.0001')
#plt.yticks(sensitivity_threshold)
#y_label=[.1,.003,0.0001])

#plt.axes.set_yticklabels('',y_label)
#ax1.set_xticklabels
#plt.Axes.axes('Frequency [GHz]')
#plt.colorbar()
ax.set_xlabel('Frequency [GHz]')
ax.set_ylabel('Sensitivity Threshold')
plt.title('Sensitivity Thresholds for Calibrator Fields')

plt.savefig('SensitivityFig-fluxonly.png')
plt.show()




# Major ticks
#ax.set_xticks(np.arange(0, 10, 1));
#ax.set_yticks(np.arange(0, 10, 1));

# Labels for major ticks
#ax.set_xticklabels(np.arange(1, 11, 1));
#ax.set_yticklabels(y_label);



In [ ]:
freq_bin=np.array(range(190,371,1))
plt.contour(np.log10(sensitivity_threshold), freq_bin,binned_array,[1,10,100])
plt.title('Contour Plot of Sensitivity Thresholds for Calibrator Fields')

plt.savefig('SensitivityContourFig-fluxonly.png')
plt.show()
#.1 is 20%, .3 is factor of 2, .5 is pi, 1 is 10

# BEGIN CONVERSION OF SENSITIVITY CALCULATIONS INTO LUMINOSITY FUNCTION



In [115]:
# Need to convert sensitivity calculations into luminosity function constraints
# Array of frequencies observed
freqObs_total=np.array(range((181)))+190
freqObs_total=freqObs_total.astype(float)


# Create new bins for sensitivity values based on freq range
#New freq observed arrays
#190 to 214 GHz
#freqObs_190=np.array(range(60))+190
#250 to 314 GHz
#freqObs_250=np.array(range(65))+250
#315 to 350 GHz
#freqObs_315=np.array(range(66))+315




In [116]:
# Derive redshift values for different spectral lines: 

# CO(3-2) @ 345 GHz, CO(4-3) @ 461 GHz, CO(5-6) @ 576 GHz, CII @ 1896 GHZ
freqEmit = np.array([345.,461.,575.,1896.])

#CO(3-2)
z_CO32_190 = (freqEmit[0]-freqObs_190)/(freqObs_190)
z_CO32_250 = (freqEmit[0]-freqObs_250)/(freqObs_250)
z_CO32_315 = (freqEmit[0]-freqObs_315)/(freqObs_315)
z_CO32_total = (freqEmit[0]-freqObs_total)/(freqObs_total)

#CO(4-3)
z_CO43_190 = (freqEmit[1]-freqObs_190)/(freqObs_190)
z_CO43_250 = (freqEmit[1]-freqObs_250)/(freqObs_250)
z_CO43_315 = (freqEmit[1]-freqObs_315)/(freqObs_315)

z_CO43_total = (freqEmit[1]-freqObs_total)/(freqObs_total)
z_CO43_total_range1=(freqEmit[1]-freqObs_total-4)/(freqObs_total-4) #want across freq space - 4 GHz
z_CO43_total_range2=(freqEmit[1]-freqObs_total+4)/(freqObs_total+4) # + 4 GHZ
#z_CO43_total_bandwidth = z_CO43_total_range2-z_CO43_total_range1 #GO BACK TO


#CO(5-6)
z_CO56_190 = (freqEmit[2]-freqObs_190)/(freqObs_190)
z_CO56_250 = (freqEmit[2]-freqObs_250)/(freqObs_250)
z_CO56_315 = (freqEmit[2]-freqObs_315)/(freqObs_315)

#CII
z_CII_190 = (freqEmit[3]-freqObs_190)/(freqObs_190)
z_CII_250 = (freqEmit[3]-freqObs_250)/(freqObs_250)
z_CII_315 = (freqEmit[3]-freqObs_315)/(freqObs_315)



In [118]:
#Calculate luminosity distance for each redshift array [Mpc]
lumdist_CO43_total = [cosmocalc(z)['DL_Mpc'] for z in z_CO43_total]

#Calculate comoving radial distance [Mpc] for total bandwidth
comdist_CO43_range1 = [cosmocalc(z)['DCMR_Mpc'] for z in z_CO43_total_range1]
comdist_CO43_range2 = [cosmocalc(z)['DCMR_Mpc'] for z in z_CO43_total_range2]
#comdist_CO43_total_bandwidth = comdist_CO43_range2-comdist_CO43_range1
#pp(lumdist)

#np.subtract(comdist_CO43_range2,comdist_CO43_range1) #GO BACK AND CHECK IF TAKE different in z vs difference in comoving length

In [119]:
# Create fucntion to calculate Brightness Temp Luminosity [L_solar]
def temp_lum(freqObs,lumDist, sv, z):
   
   c = 3.**5 #km/s
   kb = np.power(1.38,-23) #Jy/K
   return (c**2/2*kb)*np.power(freqObs_total,-2)*np.power(lumdist_CO43_total,2)*np.power((1+z),-3)*sv



In [120]:
#Set up empty structure for storing values related to brightness temperature lums
CO43_Struct={} 
Ncal = len(cal_sig_array[0]) #length of frequencies 

CO43_Struct['freq'] = np.zeros(Ncal) #freq observed
CO43_Struct['lumDistance'] = np.zeros(Ncal) # luminosity distance
CO43_Struct['Sensitivity'] = np.zeros(Ncal) # sensitivities
CO43_Struct['zRed'] = np.zeros(Ncal)  # redshift
CO43_Struct['lumTemp'] = np.zeros(Ncal)  # redshift

   

In [122]:
#find indices of sensitivities values (j corresponds to freq bin; i corresponds to source)
for i in range(len(cal_sig_array[0])): #loop over each source
    for j in range(len(freqObs_total)): #loop over each freq bin
        if cal_sig_array[j,i] != 1000000:
           # print(j,i)
            CO43_Struct['lumTemp']=temp_lum(freqObs_total, lumdist_CO43_total, cal_sig_array[j,i], z_CO32_total)
            CO43_Struct['Sensitivity']=cal_sig_array[j,i]
    
       
CO43_Struct['freq']=freqObs_total
CO43_Struct['lumDist']=lumdist_CO43_total
CO43_Struct['zRed']=z_CO43_total


In [ ]:
# Create function to calculate solid angle of beam ---> need to figure out exponential stuff
def solid_angle(freqObs):
    Aper_eff = .8
    D_dish = 6 #[m]
    A_geom = (math.pi*(D_dish**2))/4
  #####  A_geom = (D_angular*math.pi/180**2)*(math.pi*/4)
    A_eff = Aper_eff*A_geom
    c = 3*10**8 #m/s
    wavelength=c/(freqObs*10**9)
    return((wavelength**2)/A_eff)
   

print(solid_angle(345.))




# FIGURE OUT EXPONENTIAL THINGSSS

In [16]:
#Step 4.5: Figure out angular diameter of dish based on sensitivities

# the sensitivity of the telescope is better at the center of the field of view versus the edge. 
#Generally speaking, the sensitivity scales as e^(ln(0.5)*(2*x/FWHM)^2), where x is the distance from field center, 
#and the FWHM is the resolution of a single SMA telescope (roughly lambda / telescope_diameter)

def angDiam(freqObs,sens):

    D_dish = 6. #[m]
    c = 3.**8 #m/s
    wavelength=c/freqObs
    FWHM = wavelength/D_dish
    sens = math.exp(math.log(0.5)*(2*r/FWHM)**2)
    return(r)

In [80]:
freqObs=345.
D_dish = 6. #[m]
c = 3.**8 #m/s
wavelength=c/freqObs
FWHM = wavelength/D_dish
r = Symbol('r')
print(type(initial_guess))

#function = math.e**(math.log(0.5)*np.power((2*r/FWHM),2)) - .001
func1 = math.exp(math.log(0.5)*np.power(2*r/FWHM,2)) -.001
initial_guess = 5.

#r_solution= fsolve(function,initial_guess)
#print(solve(function, r))
print(100)


<class 'float'>


TypeError: can't convert expression to float

In [ ]:
# Calculate volume of space per pointing by multiplying difference in comoving radial length
# by solid angle of beam to calculate vol of space
###will then have lum func/vol of space
